In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
from mongoengine import connect
connect(
    db=os.getenv('MONGO_DB'),
    host=os.getenv('MONGO_URL'),
    port=27017,
    username=os.getenv('MONGO_USR'),
    password=os.getenv('MONGO_PWD'),
    authentication_source="admin"
    )

In [ ]:
from profile_model import ProfileModel, LocatedProfileModel
from mongoengine import connect, Document, LongField, StringField
from datetime import datetime, timedelta
import time
import pandas as pd

In [ ]:
import math
import localization as lx
# Initialize an empty dictionary to store the results

def localize(ref_points,distances):
    P=lx.Project(mode='Earth1',solver='LSE')
    if len(ref_points) < 3:
        return
    if any(math.isnan(distance) for distance in distances):
        return
    for i in range(len(ref_points)):
        P.add_anchor(f'anchore_{i}',ref_points[i])
    t,label=P.add_target(ID=123)
    for i in range(len(distances)):
        t.add_measure(f'anchore_{i}',distances[i])
    P.solve()

    return t.loc.x, t.loc.y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(locations):
    # Example reference points and distances (lat, lon) and meters
    reference_points = locations['ref_points']
    distances = locations['distances']
    calculated_point = locations['estimated_position']

    # Plotting
    fig, ax = plt.subplots()

    # Convert distances to degrees approximately (quick approximation, valid for short distances)
    # Assuming a rough conversion factor (not accurate for large distances or near the poles)
    distance_degrees = [d / 111139 for d in distances]

    # Plot reference points
    for (lat, lon), distance_degree in zip(reference_points, distance_degrees):
        ax.plot(lon, lat, 'bo')  # Reference points in blue
        circle = plt.Circle((lon, lat), distance_degree, color='b', fill=False, linestyle='--')
        ax.add_artist(circle)

    # Plot the calculated point
    ax.plot(calculated_point[1], calculated_point[0], 'rx')  # Calculated point in red

    # Set labels and show plot
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Multilateration Visualization')
    plt.grid(True)
    plt.axis('equal')  # Equal aspect ratio to ensure circles look like circles
    plt.show()

In [ ]:
current_millis = int(time.time() * 1000)

# 1 hour ago in milliseconds
one_hour_ago_millis = current_millis - 100 * (60 * 60 * 1000)

# Aggregation pipeline to filter, group by batch_timestamp, and collect profileIds
pipeline = [
    {
        "$match": {
            "batch_timestamp": {"$gt": one_hour_ago_millis}
        }
    },
    {
        "$group": {
            "_id": "$batch_timestamp",  # Grouping by batch_timestamp
            "profileIds": {"$addToSet": "$profileId"}  # Collecting unique profileIds for each batch_timestamp
        }
    },
    {
        "$sort": {"_id": 1}  # Optional: Sorting by batch_timestamp if needed
    }
]

results = ProfileModel.objects.aggregate(*pipeline)

# Processing the results to use batch_timestamp as key in a dictionary
grouped_profile_ids = {result['_id']: result['profileIds'] for result in results}

In [ ]:
locations = {}
for batch_timestamp, profile_ids in grouped_profile_ids.items():
    print(f"Batch Timestamp: {batch_timestamp}, Profile IDs: {profile_ids}")
    # Now, for each profile_id, query documents with the matching profile_id and batch_timestamp
    for profile_id in profile_ids:
        matching_documents = ProfileModel.objects(
            batch_timestamp=batch_timestamp,
            profileId=profile_id
        )
        # You can convert the QuerySet to a list of dictionaries if needed
        profiles = list(matching_documents.as_pymongo())
        # Convert the list of dictionaries to a pandas DataFrame
        profiles_df = pd.DataFrame(profiles)

        if "distanceMeters" in profiles_df.columns:
            ref_points = [[lat, lon] for lat, lon in zip(profiles_df['lat'].tolist(), profiles_df['lon'].tolist())]
            distances = profiles_df['distanceMeters'].tolist()
            estimated_position = localize(ref_points,distances)
            locations[profile_id] = {
                "estimated_position": estimated_position,
                "batch_timestamp": batch_timestamp,
                "ref_points": ref_points,
                "distances": profiles_df['distanceMeters'].tolist()
            }
            if estimated_position:
                localizedProfile = LocatedProfileModel(**profiles[0]) # Create a new LocatedProfileModel instance
                localizedProfile.lat = estimated_position[0]  # Set the estimated latitude
                localizedProfile.lon = estimated_position[1]
                localizedProfile.save()
            print(f"Profile ID: {profile_id}, Estimated Position: {estimated_position}, ref_points: {len(ref_points)}, average_distance: {sum(distances)/len(distances)}")

In [ ]:
locations.keys()

In [ ]:
visualize(locations[350730666])